In [1]:
import keras
import glob
import extract_text
import pickle
import gensim
from tqdm import tqdm_notebook as tqdm
import util

/home/charles/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [18]:
#open the files
all_files = glob.glob('files/*/*')

In [4]:
all_text = []
for f in tqdm(all_files):
    try:
        all_text.append(extract_text.apply(f))
    except:
        print("passed")
        pass

HBox(children=(IntProgress(value=0, max=433), HTML(value='')))

passed


passed
passed
Could not parse the PDF,  files\Keppel\54214cdb0cf2ce3a91b6e2bd.pdf
passed
Could not open Excel
passed
passed
Could not open Excel
passed
passed
passed
passed
passed
passed
passed
passed
Could not parse the PDF,  files\Keppel\keppel_wickens_design_and_analysis.pdf
passed
passed
passed
passed
passed
passed
Could not open Excel
passed
passed
passed
passed
passed
Could not parse the PDF,  files\Prudential\10.1007%2Fs10551-010-0597-8.pdf
WARNING *** file size (32640) not 512 + multiple of sector size (512)
passed
passed
passed
passed
passed
passed
passed
passed
passed
Could not open Excel
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed



In [2]:
#Write
#pickle.dump( all_text, open( "all_text.p", "wb" ) )
#Read
all_text = pickle.load( open( "safe/all_text.p", "rb" ) )

In [3]:
import spacy
!python -m spacy download en
nlp = spacy.load('en')


    Linking successful
    C:\Users\maxpr\Anaconda3\lib\site-packages\en_core_web_sm -->
    C:\Users\maxpr\Anaconda3\lib\site-packages\spacy\data\en

    You can now load the model via spacy.load('en')



In [ ]:
#DO NOT EXECUTE LIKE THAT
#require a lot of memory
all_docs = []
for t in tqdm(all_text):
    all_docs.append(nlp(t))
print(all_docs)

A Jupyter Widget

In [4]:
import text_prepocessing
%time sentenses, dico = text_prepocessing.apply(all_text)

100%|████████████████████████████████████████████████████████████████████████████████| 375/375 [21:00<00:00,  3.36s/it]


Wall time: 21min 15s


In [9]:
util.save(dico, 'safe/dico.p')
util.save(sentenses, 'safe/sentenses.p')

In [2]:
dico = util.load('safe/dico.p')
sentenses = util.load('safe/sentenses.p')

In [3]:
len(dico)

564576

In [5]:
dico_revers = {dico[k]:k for k in dico}

In [6]:
sentenses_word = []
for sent in sentenses:
    sent = [dico_revers[i] for i in sent]
    sentenses_word.append(sent)

In [7]:
%time model = gensim.models.Word2Vec(sentences=sentenses_word, min_count=1, iter=10, size=100)# default size is 100

Wall time: 1min 15s


In [12]:
model.wv.most_similar('prudential')#(str(dico['prudential']))

[('pru', 0.5087935924530029),
 ('company', 0.492870032787323),
 ('commitment', 0.46712547540664673),
 ('annualreport', 0.45091530680656433),
 ('sustainability', 0.4497225284576416),
 ('prei', 0.4418696165084839),
 ('prudentialfinancial', 0.4291253089904785),
 ('statement', 0.418621689081192),
 ('pfi', 0.4025071859359741),
 ('rm', 0.4003955125808716)]

In [26]:
model.corpus_count

1435

In [7]:
len(all_text)

375

In [3]:
l_total = 0
for t in tqdm(all_text):
    l_total += len(t.split())
print(l_total)

A Jupyter Widget


12172369


In [11]:
text_join = ''
for t in tqdm(all_text):
    text_join += t + '.'
len(text_join.split())

A Jupyter Widget

12172350

In [31]:
#%time doc = nlp(text_join) #make the kernel
import re
sentences = []
for t in tqdm(all_text):
    prepocessed = " ".join(re.findall("([A-Za-z@?:!]*)",t))
    
    #print(prepocessed)
    sentences.append(prepocessed) #better preprocessing should be done
#print(sentences[0])

HBox(children=(IntProgress(value=0, max=375), HTML(value='')))

In [32]:
%time model = gensim.models.Word2Vec(sentences=sentences, min_count=1, iter=10, size=100)# default size is 100

Wall time: 37.4 s


In [23]:
model.vector_size

100

In [48]:
model.wv.similarity('Keppel', 'Prudential')

0.6170042476875606

The similar words are good but clearly some preprocessing needs to be done